In [1]:
%cd ~/ipython-notebook/dqn4wirelesscontrol/
import sys
import time
import pandas as pd
import lasagne
pd.set_option('mode.chained_assignment', None)  # block warnings due to DataFrame value assignment
# Project modules
from sleep_control.integration import Emulation
import sleep_control.traffic_emulator
import sleep_control.traffic_server
from sleep_control.controller import QController, DummyController
import qlearning.qtable
import qlearning.qnn
from qlearning.qtable import QAgent
from qlearning.qnn import QAgentNN
from qlearning.mixin import PhiMixin
std_out = sys.stdout
class QAgentNNPhi(PhiMixin, QAgentNN):
    def __init__(self, **kwargs):
        super(QAgentNNPhi, self).__init__(**kwargs)

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


/home/admin-326/Data/ipython-notebook/dqn4wirelesscontrol


In [2]:
sys_stdout = std_out

# Setting-up data

In [3]:
session_df = pd.read_csv(filepath_or_buffer='./sleep_control/data/net_traffic_nonull.dat', 
                         sep=',', 
                         names=['uid','location','startTime_unix','duration_ms','domainProviders','domainTypes','domains','bytesByDomain','requestsByDomain'])
session_df.index.name = 'sessionID'
session_df['endTime_unix'] = session_df['startTime_unix'] + session_df['duration_ms']
session_df['startTime_datetime'] = pd.to_datetime(session_df['startTime_unix'], unit='ms')  # convert start time to readible date_time strings
session_df['endTime_datetime'] = pd.to_datetime(session_df['endTime_unix'], unit='ms')
session_df['totalBytes'] = session_df['bytesByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum bytes across domains
session_df['totalRequests'] = session_df['requestsByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum requests across domains
session_df.sort(['startTime_datetime'], ascending=True, inplace=True)  # get it sorted
session_df['interArrivalDuration_datetime'] = session_df.groupby('location')['startTime_datetime'].diff()  # group-wise diff
session_df['interArrivalDuration_ms'] = session_df.groupby('location')['startTime_unix'].diff()  # group-wise diff
session_df = session_df[session_df['location']=='第三食堂']

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


---

In [7]:
#1 BUF1 Start=2014-09-25 09:20:31.474000, test if a single buffer will stuck
gamma, alpha = 0.9, 0.9
explore_strategy, epsilon = 'epsilon', 0.02
reward_scaling, reward_scaling_update, rs_period = 1, 'adaptive', 32
phi_length = 5
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
momentum, learning_rate, update_period = 0.9, 0.01, 16
num_buffer, memory_size, batch_size, freeze_period  = 1, 200, 100, 4
log_file = "sleep_control/experiments/log/message_2016-6-9_1315_BUF1_G9.log"
time_step, n_days = pd.Timedelta(seconds=2), 7
TOTAL_EPOCHS = 3600*24*n_days/2
backoff = num_buffer*memory_size+phi_length
head_datetime = pd.to_datetime("2014-09-25 09:20:31.474000") # - time_step*backoff

In [8]:
log_file = open(log_file,"w")
dim_state = (1, phi_length, 3+2)
actions = [(True, None), (False, 'serve_all')]
tail_datetime = head_datetime + pd.Timedelta(days=1)*n_days*5
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df, 
                                                    head_datetime=head_datetime, tail_datetime=tail_datetime, 
                                                    time_step=time_step, verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
agent = QAgentNNPhi(phi_length=phi_length, dim_state=dim_state, range_state=[[range_state_slice]*phi_length], actions=actions,
        alpha=alpha, learning_rate=learning_rate, momentum=momentum, update_period=update_period,
        reward_scaling=reward_scaling, reward_scaling_update=reward_scaling_update,
        memory_size=memory_size, num_buffer=num_buffer, freeze_period=freeze_period, batch_size=batch_size,
        gamma=gamma, explore_strategy=explore_strategy, epsilon=epsilon,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)
t = time.time()
sys.stdout = log_file
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdout = std_out
log_file.close()
time.time()-t

New TrafficEmulator with parameters:
  head=2014-09-25 09:20:31.474000
  tail=2014-10-30 09:20:31.474000
  time_step=0 days 00:00:02
  epoch=0
  verbose=2


3635.799090862274

---

In [4]:
#2 Fixed RS tests
gamma, alpha = 0.5, 0.9
explore_strategy, epsilon = 'epsilon', 0.02
reward_scaling, reward_scaling_update, rs_period = 1000, 'fixed', 32
phi_length = 5
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
momentum, learning_rate, update_period = 0.9, 0.01, 16
num_buffer, memory_size, batch_size, freeze_period  = 2, 200, 100, 4
log_file = "sleep_control/experiments/log/message_2016-6-11_1230_FR1000_G5.log"
time_step, n_days = pd.Timedelta(seconds=2), 7
TOTAL_EPOCHS = 3600*24*n_days/2
backoff = num_buffer*memory_size+phi_length
head_datetime = pd.to_datetime("2014-09-25 09:20:31.474000") # - time_step*backoff

In [6]:
log_file = open(log_file,"w")
dim_state = (1, phi_length, 3+2)
actions = [(True, None), (False, 'serve_all')]
tail_datetime = head_datetime + pd.Timedelta(days=1)*n_days*5
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df, 
                                                    head_datetime=head_datetime, tail_datetime=tail_datetime, 
                                                    time_step=time_step, verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
agent = QAgentNNPhi(phi_length=phi_length, dim_state=dim_state, range_state=[[range_state_slice]*phi_length], actions=actions,
        alpha=alpha, learning_rate=learning_rate, momentum=momentum, update_period=update_period,
        reward_scaling=reward_scaling, reward_scaling_update=reward_scaling_update,
        memory_size=memory_size, num_buffer=num_buffer, freeze_period=freeze_period, batch_size=batch_size,
        gamma=gamma, explore_strategy=explore_strategy, epsilon=epsilon,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)
t = time.time()
sys.stdout = log_file
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdout = std_out
log_file.close()
time.time()-t

TypeError: coercing to Unicode: need string or buffer, file found